In [3]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

# Load JSON files
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

vetting_playlist = load_json('Vetting_Playlist.json')
songs = load_json('songs_0-999.json')
artists = load_json('artists_0-999.json')
albums = load_json('albums_0-999.json')



In [4]:
import pandas as pd

# Assuming songs_data is your loaded JSON data
songs_data = load_json('songs_0-999.json')

# Extract song details including artist names and album name
song_details = []
for song_id, details in songs_data.items():
    artist_names = ', '.join([artist['name'] for artist in details['album']['artists']])
    album_name = details['album']['name']
    song_details.append({'song_id': song_id, 'artist_names': artist_names, 'album_name': album_name})

# Convert to DataFrame
df_songs = pd.DataFrame(song_details)


In [6]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

# Function to load JSON data
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Load songs data
songs_data = load_json('songs_0-999.json')


In [8]:
# Extract song details
song_details = []
for song_id, details in songs_data.items():
    artist_names = ', '.join([artist['name'] for artist in details['album']['artists']])
    album_name = details['album']['name']
    song_details.append({'song_id': song_id, 'artist_names': artist_names, 'album_name': album_name})

# Convert to DataFrame
df_songs = pd.DataFrame(song_details)

In [9]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df_songs['artist_names'] + " " + df_songs['album_name'])


In [11]:
kmeans = KMeans(n_clusters=10, random_state=42)  # Adjust the number of clusters as needed
df_songs['cluster'] = kmeans.fit_predict(tfidf_matrix)


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [21]:
vetting_playlist_songs = vetting_playlist_data.get('tracks', {}).get('items', [])

vetting_playlist_details = []
for song_entry in vetting_playlist_songs:
    try:
        # Check if 'track' key exists and contains needed data
        if 'track' in song_entry and 'album' in song_entry['track']:
            track_data = song_entry['track']
            song_id = track_data.get('id', 'Unknown ID')  # Providing a fallback value
            artist_names = ', '.join(artist.get('name', 'Unknown Artist') for artist in track_data['album']['artists'])
            album_name = track_data['album'].get('name', 'Unknown Album')
            vetting_playlist_details.append({
                'song_id': song_id,
                'artist_names': artist_names,
                'album_name': album_name
            })
        else:
            print(f"Missing 'album' data for song entry: {song_entry.get('track', {}).get('id', 'Unknown ID')}")
    except Exception as e:
        print(f"Error processing song entry: {e}")

df_vetting_playlist = pd.DataFrame(vetting_playlist_details)


In [22]:
# Display the first few rows of the vetting playlist DataFrame to verify its contents
print(df_vetting_playlist.head())

                  song_id        artist_names     album_name
0  51ggEZWbFP5dMdWjGCDYTk      Alabama Shakes  Sound & Color
1  6p5LBZkLRIeVhbLh0w7EuP      Alabama Shakes  Future People
2  2lrRWbiHPdBicufAqrGN3r      Alabama Shakes  Sound & Color
3  7Gz1adaLeIHNOqJPvfnncD      Alabama Shakes  Sound & Color
4  17ZnveSDBpG9QtL7zLJNPy  Heartless Bastards          Arrow


In [23]:
# Find the most common artists in the vetting playlist
common_artists = df_vetting_playlist['artist_names'].value_counts()

print("Most Common Artists in Vetting Playlist:")
print(common_artists.head())  # Adjust the number passed to head() as desired


Most Common Artists in Vetting Playlist:
Angel Olsen       7
Alabama Shakes    6
Shakey Graves     4
Big Thief         4
DeVotchKa         3
Name: artist_names, dtype: int64


In [27]:
# Function to load JSON data
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Load another songs dataset
other_songs_data = load_json('songs_0-999.json')

# Extract song details from the other dataset
other_songs_details = []
for song_id, details in other_songs_data.items():
    try:
        artist_names = ', '.join(artist['name'] for artist in details['album']['artists'])
        album_name = details['album']['name']
        other_songs_details.append({
            'song_id': song_id,
            'artist_names': artist_names,
            'album_name': album_name
        })
    except KeyError:
        print(f"KeyError processing song ID {song_id}")

# Convert to DataFrame
df_other_songs = pd.DataFrame(other_songs_details)


In [28]:
df_vetting_playlist['combined'] = df_vetting_playlist['artist_names'] + " " + df_vetting_playlist['album_name']
df_other_songs['combined'] = df_other_songs['artist_names'] + " " + df_other_songs['album_name']


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Create TF-IDF vectors for all songs combined
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix_all = tfidf_vectorizer.fit_transform(df_vetting_playlist['combined'].tolist() + df_other_songs['combined'].tolist())

# Calculate cosine similarities
similarity_matrix = cosine_similarity(tfidf_matrix_all[:len(df_vetting_playlist)], tfidf_matrix_all[len(df_vetting_playlist):])

# Find the most similar song for each song in the vetting playlist
most_similar_indices = similarity_matrix.argmax(axis=1)
most_similar_scores = similarity_matrix.max(axis=1)


In [30]:
# Extract the most similar songs' details
similar_songs_details = df_other_songs.iloc[most_similar_indices].reset_index(drop=True)
similar_songs_details['similarity_score'] = most_similar_scores

# Add titles from the vetting playlist for reference
similar_songs_details['vetting_song_id'] = df_vetting_playlist['song_id'].values

# Display the result
print(similar_songs_details[['vetting_song_id', 'song_id', 'artist_names', 'album_name', 'similarity_score']])


           vetting_song_id                 song_id        artist_names  \
0   51ggEZWbFP5dMdWjGCDYTk  2lrRWbiHPdBicufAqrGN3r      Alabama Shakes   
1   6p5LBZkLRIeVhbLh0w7EuP  26ZTwQyRVbec9wQ0OaihjE      Alabama Shakes   
2   2lrRWbiHPdBicufAqrGN3r  2lrRWbiHPdBicufAqrGN3r      Alabama Shakes   
3   7Gz1adaLeIHNOqJPvfnncD  2lrRWbiHPdBicufAqrGN3r      Alabama Shakes   
4   17ZnveSDBpG9QtL7zLJNPy  17ZnveSDBpG9QtL7zLJNPy  Heartless Bastards   
..                     ...                     ...                 ...   
95  0DzNSw9Pp1yKJZynNBM8uy  6v3QCw4ca5cR4VkzpG0PHy           Alkilados   
96  6qZjm61s6u8Ead9sWxCDro  6vTtMyCg96xwpoIBws9K0Q         Tame Impala   
97  5mqr5kF7oNDdcG01qazSqK  7gxakgZXtvTcBjaxOAZ08G      Cherry Glazerr   
98  1rO4Ew8dCbYWqJ3L6DmJDo  5E5HYgxGMp3BPakHGfKfIB   The Modern Lovers   
99  4lZjHEJXzKCCTVPtgYcypv  7nSyjL3LIXy2DXF9003HdX        Thee Oh Sees   

                              album_name  similarity_score  
0                          Sound & Color          

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Assuming `combined_texts` contains all your combined artist and album names
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_texts)

distortions = []
K = range(1, 11)  # Adjust the range of k as needed
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(tfidf_matrix)
    distortions.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(K, distortions, 'bx-')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()


NameError: name 'combined_texts' is not defined